In [3]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

--2024-04-26 05:25:55--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  40.5MB/s    in 1.5s    

2024-04-26 05:25:57 (40.5 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 21 19:15:43 2024
@author: wxdycq
"""

import cv2
import dlib
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import Callback
import shutil
import random
import os
import sys

# extract data
isColab = "google.colab" in sys.modules

if isColab:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    base_path = "/content/drive/My Drive/"
    train_dir = os.path.join(base_path, "train")
    test_dir = os.path.join(base_path, "test 2")

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# transfer tranning data to be fit in CNN (all infor)
def extract_features(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    features = {}

    for face in faces:
        landmarks = predictor(gray, face)
        for (start, end, name) in [(36, 42, "left_eye"), (42, 48, "right_eye"), (48, 68, "mouth")]:
            points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(start, end)])
            x, y, w, h = cv2.boundingRect(points)
            feature_img = img[y:y+h, x:x+w]
            features[name] = Image.fromarray(feature_img)

        return features

# Build CNN model
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# process image data (change shape)
def prepare_image(image):
    image = image.resize((50, 50))
    image_array = np.array(image) / 255.0
    return np.expand_dims(image_array, axis=0)

# generated the final data
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=40, width_shift_range=0.2,
    height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(50, 50),  # Ensure this matches the input shape expected by the model
    batch_size=64,
    class_mode='categorical'  # 'binary' or 'categorical' based on your need
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(50, 50),  # Ensure this matches the input shape expected by the model
    batch_size=64,
    class_mode='categorical'  # 'binary' or 'categorical' based on your need
)

# Model infor, like accuracy, loss
class MetricsCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Starting epoch {epoch+1}")

    def on_batch_end(self, batch, logs=None):
        print(f"After batch {batch+1}:")
        print(f" - batch loss: {logs['loss']:.4f}")
        print(f" - batch accuracy: {logs['accuracy']:.4f}")

    def on_epoch_end(self, epoch, logs=None):
        print(f"After epoch {epoch+1}:")
        print(f" - epoch train loss: {logs['loss']:.4f}")
        print(f" - epoch train accuracy: {logs['accuracy']:.4f}")
        if 'val_loss' in logs:
            print(f" - epoch validation loss: {logs['val_loss']:.4f}")
        if 'val_accuracy' in logs:
            print(f" - epoch validation accuracy: {logs['val_accuracy']:.4f}")

# Build and train the model (call function)

model = build_model()
model.summary()  # This will print the output shapes of each layer
try:
    model.fit(
        train_generator,
        epochs=10,
        callbacks=[MetricsCallback()],
        validation_data=test_generator  # Make sure there is data here!
    )
except Exception as e:
    print("An error occurred during model training:", e)

# test model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")



Mounted at /content/drive
Found 6038 images belonging to 2 classes.
Found 1419 images belonging to 2 classes.
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 48, 48, 32)        896       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 24, 24, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 22, 22, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 11, 11, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 9, 9, 64)          36928     
          